In [73]:
import pandas as pd
from pyspark.mllib.clustering import KMeans, KMeansModel
import numpy as np
from numpy import array

# Import Data

In [21]:
# add clicks
adclicksDF = pd.read_csv('./big_data_capstone_datasets_and_scripts/flamingo-data/ad-clicks.csv')
adclicksDF = adclicksDF.rename(columns=lambda x: x.strip()) #remove whitespaces from headers

In [ ]:
# buy clicks
buyclicksDF = pd.read_csv('./big_data_capstone_datasets_and_scripts/flamingo-data/buy-clicks.csv')
buyclicksDF = buyclicksDF.rename(columns=lambda x: x.strip()) #removes whitespaces from

In [38]:
# game clicks
gameclicksDF = pd.read_csv('./big_data_capstone_datasets_and_scripts/flamingo-data/game-clicks.csv')
gameclicksDF = gameclicksDF.rename(columns=lambda x: x.strip()) #removes whitespaces from

In [41]:
# user session
usersessionDF = pd.read_csv('./big_data_capstone_datasets_and_scripts/flamingo-data/user-session.csv')
usersessionDF = usersessionDF.rename(columns=lambda x: x.strip()) #removes whitespaces from

# Display Data

In [25]:
adclicksDF.head(n=5)

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory,adCount
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics,1
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies,1
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers,1
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion,1
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing,1


In [39]:
buyclicksDF.head(n=5)

,timestamp,txId,userSessionId,team,userId,buyId,price
0,2016-05-26 15:36:54,6004,5820,9,1300,2,3.0
1,2016-05-26 15:36:54,6005,5775,35,868,4,10.0
2,2016-05-26 15:36:54,6006,5679,97,819,5,20.0
3,2016-05-26 16:36:54,6067,5665,18,121,2,3.0
4,2016-05-26 17:06:54,6093,5709,11,2222,5,20.0


In [43]:
gameclicksDF.head()

,timestamp,clickId,userId,userSessionId,isHit,teamId,teamLevel
0,2016-05-26 15:06:55,105,1038,5916,0,25,1
1,2016-05-26 15:07:09,154,1099,5898,0,44,1
2,2016-05-26 15:07:14,229,899,5757,0,71,1
3,2016-05-26 15:07:14,322,2197,5854,0,99,1
4,2016-05-26 15:07:20,22,1362,5739,0,13,1


In [42]:
usersessionDF.head(n=5)

,timestamp,userSessionId,userId,teamId,assignmentId,sessionType,teamLevel,platformType
0,2016-05-26 14:47:30,5675,170,90,5423,start,1,iphone
1,2016-05-27 00:06:54,5675,170,90,5423,end,1,iphone
2,2016-05-26 14:43:09,5795,835,39,5300,start,1,android
3,2016-05-27 08:06:54,5795,835,39,5300,end,1,android
4,2016-05-26 14:47:20,5833,624,2,5529,start,1,iphone


# Feature Selection

In [28]:
userPurchases = buyclicksDF[['userId','price']] #select only userid and price
userPurchases.head(n=5)

,userId,price
0,1300,3.0
1,868,10.0
2,819,20.0
3,121,3.0
4,2222,20.0


In [44]:
useradClicks = adclicksDF[['userId', 'adCount']]
useradClicks.head(n=5) #as we saw before, this line displays first five lines

,userId,adCount
0,611,1
1,1874,1
2,2139,1
3,212,1
4,1027,1


In [45]:
gameClicks = gameclicksDF[['userId', 'isHit']]
gameClicks.head()

,userId,isHit
0,1038,0
1,1099,0
2,899,0
3,2197,0
4,1362,0


In [92]:
sessionHours = usersessionDF[['userId', 'userSessionId', 'timestamp', 'sessionType']]
sessionHours.head()

,userId,userSessionId,timestamp,sessionType
0,170,5675,2016-05-26 14:47:30,start
1,170,5675,2016-05-27 00:06:54,end
2,835,5795,2016-05-26 14:43:09,start
3,835,5795,2016-05-27 08:06:54,end
4,624,5833,2016-05-26 14:47:20,start


In [93]:
sessionHours['timestamp'] = pd.to_datetime(sessionHours['timestamp'])
startSession = sessionHours[sessionHours['sessionType'] == 'start']
endSession = sessionHours[sessionHours['sessionType'] == 'end']
sessionHours = startSession.merge(endSession, on='userSessionId')
sessionHours['sessionLength'] = sessionHours['timestamp_y'] - sessionHours['timestamp_x']
sessionHours['sessionLength'] = sessionHours['sessionLength'] / np.timedelta64(1, 'h')

In [94]:
sessionHours = sessionHours[['userId_x', 'sessionLength']]

In [95]:
sessionHours.columns = ['userId', 'sessionLength']

In [96]:
sessionHours.head()

,userId,sessionLength
0,170,9.323333
1,835,17.395833
2,624,32.326111
3,514,32.489167
4,2103,32.499722


# Aggregation

In [33]:
# aggregate ads count by users and addtype
adsPerUser = useradClicks.groupby(['userId']).sum()
adsPerUser = adsPerUser.reset_index()
adsPerUser.columns = ['userId', 'totalAdClicks'] #rename the columns

In [34]:
adsPerUser.head(n=5)

,userId,totalAdClicks
0,1,44
1,8,10
2,9,37
3,10,19
4,12,46


In [36]:
# aggregate revenue by users
revenuePerUser = userPurchases.groupby('userId').sum()
revenuePerUser = revenuePerUser.reset_index()
revenuePerUser.columns = ['userId', 'revenue'] #rename the columns

In [37]:
revenuePerUser.head(n=5)

,userId,revenue
0,1,21.0
1,8,53.0
2,9,80.0
3,10,11.0
4,12,215.0


In [46]:
# aggregate game clicks by users
hitsPerUser = gameClicks.groupby('userId').sum()
hitsPerUser = hitsPerUser.reset_index()
hitsPerUser.columns = ['userId', 'hits']

In [47]:
hitsPerUser.head()

,userId,hits
0,0,143
1,1,96
2,2,22
3,6,16
4,8,38


In [98]:
# aggegrate total session hours by users
sessionHours = sessionHours.groupby('userId').sum()
sessionHours = sessionHours.reset_index()
sessionHours.columns = ['userId', 'sessionLength']

In [99]:
sessionHours.head()

,userId,sessionLength
0,0,175.500000
1,1,437.615278
2,2,319.500000
3,6,74.500000
4,8,80.000000


In [100]:
sessionHits = sessionHours.merge(hitsPerUser, on='userId') #userid, adCount, price

In [102]:
sessionHits['hitsPerHour'] = sessionHits['hits'] / sessionHits['sessionLength']

In [103]:
sessionHits.head()

,userId,sessionLength,hits,hitsPerHour
0,0,175.500000,143,0.814815
1,1,437.615278,96,0.219371
2,2,319.500000,22,0.068858
3,6,74.500000,16,0.214765
4,8,80.000000,38,0.475000


In [104]:
sessionHitsPerUser = sessionHits[['userId', 'hitsPerHour']]

In [105]:
sessionHitsPerUser.head()

,userId,hitsPerHour
0,0,0.814815
1,1,0.219371
2,2,0.068858
3,6,0.214765
4,8,0.475000


# Merge

In [106]:
combinedDF = adsPerUser.merge(revenuePerUser, on='userId') #userid, adCount, price

In [107]:
combinedDF = combinedDF.merge(sessionHitsPerUser, on='userId') # hitsPerHour

In [108]:
combinedDF.head(n=5) #display how the merged table looks

,userId,totalAdClicks,revenue,hitsPerHour
0,1,44,21.0,0.219371
1,8,10,53.0,0.475000
2,9,37,80.0,0.164894
3,10,19,11.0,1.333333
4,12,46,215.0,0.221420


In [109]:
trainingDF = combinedDF[['totalAdClicks','revenue', 'hitsPerHour']]

In [115]:
trainingDF.head(n=5)

,totalAdClicks,revenue,hitsPerHour
0,44,21.0,0.219371
1,10,53.0,0.475000
2,37,80.0,0.164894
3,19,11.0,1.333333
4,46,215.0,0.221420


In [116]:
trainingDF.shape

(529, 3)

In [118]:
pDF = sqlContext.createDataFrame(trainingDF)

In [119]:
parsedData = pDF.rdd.map(lambda line: array([line[0], line[1], line[2]])) #totalAdClicks, revenue

# Train KMeans model

In [127]:
my_kmmodel = KMeans.train(parsedData, 3, maxIterations=10, runs=10, initializationMode="random")

In [128]:
print(my_kmmodel.centers)

[array([  41.06666667,  145.51111111,    0.30275967]), array([ 27.14044944,  17.07022472,   0.46141664]), array([ 34.3203125 ,  66.78125   ,   0.40095501])]
